In [1]:
import matplotlib as mpl 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd 
import os 
import sys
import time 
import tensorflow as tf 
from tensorflow import keras 

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [2]:
# 导入数据
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state=7)

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state= 11)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape)
print(x_valid_scaled.shape)
print(x_test_scaled.shape)
print(np.max(x_train), np.min(x_train), np.max(x_train_scaled), np.min(x_train_scaled))

(11610, 8)
(3870, 8)
(5160, 8)
28566.0 -124.3 81.80221745207753 -2.3594398206242224


In [8]:
# metric使用
metric = keras.metrics.MeanSquaredError() 
print(metric([5.], [2.]))
print(metric([0.], [1.]))

print(metric.result())
metric.reset_states()
metric([1.], [3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [11]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch结束 验证集 metric
epochs = 100
batch_size = 32 
steps_per_epoch = len(x_train_scaled) //batch_size 
optimizer = keras.optimizers.SGD() 
metric = keras.metrics.MeanSquaredError()

def random_batch(x, y, batch_size = 32):
    idx = np.random.randint(0, len(x), size = batch_size)
    return x[idx], y[idx]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                      input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])

for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch= random_batch(x_train_scaled, y_train, batch_size)
        
        with tf.GradientTape() as tape:
            
            y_pred = model(x_batch)
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch", epoch, " train mse:", metric.result().numpy(), end="")
        
    y_valid_pred = model(x_valid_scaled)
    valid_loss = tf.reduce_mean(
        keras.losses.mean_squared_error(y_valid_pred, y_valid))
    
    print("\t", "valid mse:", valid_loss.numpy())
    
        

Epoch 0  train mse: 2.9509854	 valid mse: 1.4660633839570008
Epoch 1  train mse: 1.619361	 valid mse: 1.410769448033064
Epoch 2  train mse: 1.290606	 valid mse: 1.396533582321529
Epoch 3  train mse: 1.2928342	 valid mse: 1.39362948775909
Epoch 4  train mse: 1.2926458	 valid mse: 1.3902419818740461
Epoch 5  train mse: 1.2543572	 valid mse: 1.3914491509318812
Epoch 6  train mse: 1.2564406	 valid mse: 1.388019769660402
Epoch 7  train mse: 1.2579564	 valid mse: 1.3989449932493254
Epoch 8  train mse: 1.3025923	 valid mse: 1.3868204917438705
Epoch 9  train mse: 1.2809137	 valid mse: 1.3876654743763983
Epoch 10  train mse: 1.2750435	 valid mse: 1.3863944587948565
Epoch 11  train mse: 1.242572	 valid mse: 1.390435079743571
Epoch 12  train mse: 1.294721	 valid mse: 1.3862112432559874
Epoch 13  train mse: 1.2581574	 valid mse: 1.394424819064982
Epoch 14  train mse: 1.2214607	 valid mse: 1.3872408463665218
Epoch 15  train mse: 1.2398424	 valid mse: 1.3848889110081715
Epoch 16  train mse: 1.250680